In [3]:
#Read in the data


import numpy as np
import pandas as pd
import scipy
import networkx as nx

data = pd.read_csv("Mars_Nodes_Weights.csv", header=None)
data = data.iloc[1:]

In [4]:
#Create the Graph and populate it with the nodes and edge weights

G = nx.DiGraph()

edgeList = data.values.tolist()

for i in range(len(edgeList)):
    G.add_edge(str(edgeList[i][0]), str(edgeList[i][1]), weight= int(edgeList[i][2]))  
    
#Adding edges automatically adds the relevent nodes as well

In [5]:
#Lets look at our nodes and edges

list(G.nodes)

['rover',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S']

In [6]:
list(G.edges)

[('rover', 'A'),
 ('rover', 'B'),
 ('rover', 'C'),
 ('A', 'B'),
 ('A', 'D'),
 ('A', 'E'),
 ('B', 'A'),
 ('B', 'D'),
 ('B', 'E'),
 ('B', 'F'),
 ('B', 'C'),
 ('C', 'B'),
 ('C', 'E'),
 ('C', 'F'),
 ('D', 'G'),
 ('D', 'H'),
 ('D', 'E'),
 ('E', 'D'),
 ('E', 'G'),
 ('E', 'H'),
 ('E', 'I'),
 ('E', 'F'),
 ('F', 'E'),
 ('F', 'H'),
 ('F', 'I'),
 ('G', 'J'),
 ('G', 'H'),
 ('G', 'K'),
 ('H', 'G'),
 ('H', 'J'),
 ('H', 'K'),
 ('H', 'L'),
 ('H', 'I'),
 ('I', 'H'),
 ('I', 'K'),
 ('I', 'L'),
 ('J', 'M'),
 ('J', 'N'),
 ('J', 'K'),
 ('K', 'J'),
 ('K', 'M'),
 ('K', 'N'),
 ('K', 'O'),
 ('K', 'L'),
 ('L', 'K'),
 ('L', 'N'),
 ('L', 'O'),
 ('M', 'P'),
 ('M', 'Q'),
 ('M', 'N'),
 ('N', 'M'),
 ('N', 'P'),
 ('N', 'Q'),
 ('N', 'R'),
 ('N', 'O'),
 ('O', 'N'),
 ('O', 'Q'),
 ('O', 'R'),
 ('P', 'S'),
 ('P', 'Q'),
 ('Q', 'P'),
 ('Q', 'S'),
 ('Q', 'R'),
 ('R', 'Q'),
 ('R', 'S')]

In [9]:
G['A']

AtlasView({'B': {'weight': 18}, 'D': {'weight': 6}, 'E': {'weight': 6}})

In [43]:
nx.algorithms.shortest_paths.weighted.dijkstra_path(G, 'rover', 'P')

['rover', 'C', 'E', 'H', 'J', 'N', 'P']

In [51]:
import heapq

def dijkstra(G, start, end):
    search = [(0, start)]
    path = {start: None}
    cost = {start: 0}

    while search:
        c, n = heapq.heappop(search)
        
        if n == end:
            shortest_path = [end]
            while path[n] is not None:
                shortest_path.append(path[n])
                n = path[n]
            return (list(reversed(shortest_path)), cost[end])
        
        for nbr in G[n]:
            w = G[n][nbr]['weight']
            if nbr not in path or (nbr in path and cost[nbr] > c + w):
                path[nbr] = n
                cost[nbr] = c + w
                heapq.heappush(search, (c + w, nbr))

In [52]:
dijkstra(G, 'rover', 'P')

(['rover', 'C', 'E', 'H', 'J', 'N', 'P'], 12)

In [77]:
def a_star(G, start, end, h):
    search = [(0 + h(G, start, end), start)]
    path = {start: None}
    cost = {start: 0}

    while search:
        c, n = heapq.heappop(search)
        
        if n == end:
            shortest_path = [end]
            while path[n] is not None:
                shortest_path.append(path[n])
                n = path[n]
            return (list(reversed(shortest_path)), cost[end])
        
        for nbr in G[n]:
            w = G[n][nbr]['weight']
            if nbr not in path or (nbr in path and cost[nbr] > cost[n] + w):
                path[nbr] = n
                cost[nbr] = cost[n] + w
                heapq.heappush(search, (c + w + h(G, nbr, end), nbr))

In [83]:
def heuristic(G, a, b):
    if a == 'rover':
        ord_a = ord('A') - 1
    else:
        ord_a = ord(a)
        
    return (ord(b) - ord_a) / 4

In [84]:
a_star(G, 'rover', 'P', heuristic)

(['rover', 'C', 'E', 'H', 'K', 'N', 'P'], 12)

In [85]:
%%timeit

dijkstra(G, 'rover', 'P')

191 µs ± 31.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [86]:
%%timeit

a_star(G, 'rover', 'P', heuristic)

186 µs ± 14.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
